In [16]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import os

# ========== Parameter ==========
FREQ = 52                   # For annualization, use 52 periods per year
TARGET_MEAN_ANNUAL = 0.20   # targeted annualized mean
DATA_PATH = "spx_returns_weekly.xlsx" 
RISK_FREE = 0.0             # risk free
LOWER, UPPER = -0.20, 0.35  # constrain the weights

In [17]:
# ========== Load Data ==========

selected_tickers = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AVGO', 'BRK/B', 'LLY']

df_spx = pd.read_excel(DATA_PATH, sheet_name="s&p500 rets")
if 'date' in df_spx.columns:
    df_spx = df_spx.set_index('date')

cols_spx = [c for c in df_spx.columns if c in selected_tickers]
df_spx = df_spx[cols_spx]

df_bench = pd.read_excel(DATA_PATH, sheet_name="benchmark rets")
if 'date' in df_bench.columns:
    df_bench = df_bench.set_index('date')
df_bench = df_bench[['SPY']]

df = pd.concat([df_spx, df_bench], axis=1, join="inner")


tickers = df.columns.tolist()
n = len(tickers)
print(tickers)

['AAPL', 'AMZN', 'AVGO', 'BRK/B', 'GOOGL', 'LLY', 'META', 'MSFT', 'NVDA', 'TSLA', 'SPY']


In [18]:
df

,AAPL,AMZN,AVGO,BRK/B,GOOGL,LLY,META,MSFT,NVDA,TSLA,SPY
date,,,,,,,,,,,
2015-01-09,0.024514,-0.037534,0.047971,0.002011,-0.054445,-0.001855,-0.009055,0.009195,-0.009315,-0.057685,-0.005744
2015-01-16,-0.053745,-0.020880,-0.010268,-0.001739,0.019448,0.010726,-0.032931,-0.020131,0.000836,-0.065760,-0.012827
2015-01-23,0.065950,0.074431,0.030500,-0.000603,0.061685,0.020514,0.035255,0.020329,0.037578,0.042575,0.016565
2015-01-30,0.036997,0.134900,-0.038331,-0.034938,-0.008130,-0.001802,-0.024669,-0.143706,-0.072636,0.011476,-0.026931
2015-02-06,0.019114,0.055737,0.018037,0.043569,-0.006812,-0.022778,-0.018967,0.049753,0.062269,0.067589,0.030584
...,...,...,...,...,...,...,...,...,...,...,...
2025-04-25,0.062442,0.094896,0.124686,0.024604,0.071448,0.053074,0.091310,0.065447,0.093802,0.180553,0.046029
2025-05-02,-0.018778,0.005238,0.058915,0.016649,0.012784,-0.068872,0.090906,0.110833,0.031439,0.007931,0.029275
2025-05-09,-0.033212,0.016212,0.022392,-0.048277,-0.068772,-0.108120,-0.007588,0.007926,0.018777,0.038474,-0.004270


In [20]:
# ========== Calculate Annual Mean and Annual Cov ==========
mean_weekly = df.mean()
cov_weekly = df.cov()

mean_annual = mean_weekly * FREQ
cov_annual = cov_weekly * FREQ

target_mean_weekly = TARGET_MEAN_ANNUAL / FREQ

print("---------------------------Annual Mean---------------------------")
print(mean_annual)
print("---------------------------Annual Cov---------------------------")
print(cov_annual)

---------------------------Annual Mean---------------------------
AAPL     0.238714
AMZN     0.293447
AVGO     0.394854
BRK/B    0.135025
GOOGL    0.216800
LLY      0.281542
META     0.261924
MSFT     0.261402
NVDA     0.645580
TSLA     0.469754
SPY      0.131264
dtype: float64
---------------------------Annual Cov---------------------------
           AAPL      AMZN      AVGO     BRK/B     GOOGL       LLY      META  \
AAPL   0.076524  0.040962  0.052996  0.021636  0.042425  0.018447  0.041728   
AMZN   0.040962  0.093636  0.045583  0.017120  0.050690  0.013964  0.055302   
AVGO   0.052996  0.045583  0.140724  0.023599  0.047179  0.016347  0.050385   
BRK/B  0.021636  0.017120  0.023599  0.036349  0.019575  0.015668  0.019914   
GOOGL  0.042425  0.050690  0.047179  0.019575  0.078317  0.015148  0.051943   
LLY    0.018447  0.013964  0.016347  0.015668  0.015148  0.080078  0.016052   
META   0.041728  0.055302  0.050385  0.019914  0.051943  0.016052  0.123437   
MSFT   0.039174  0.04539

In [21]:
# ========== Single-asset Annual Statistics ==========
asset_annual_std = np.sqrt(np.diag(cov_annual)) 
asset_sharpes = (mean_annual - RISK_FREE) / asset_annual_std
asset_summary = pd.DataFrame({
    'mean_annual': mean_annual,
    'std_annual': asset_annual_std,
    'sharpe': asset_sharpes
}).sort_values('sharpe', ascending=False)

asset_summary

,mean_annual,std_annual,sharpe
NVDA,0.645580,0.463283,1.393490
MSFT,0.261402,0.239980,1.089266
AVGO,0.394854,0.375132,1.052574
LLY,0.281542,0.282980,0.994919
AMZN,0.293447,0.306000,0.958976
AAPL,0.238714,0.276629,0.862938
TSLA,0.469754,0.586431,0.801039
GOOGL,0.216800,0.279851,0.774698
SPY,0.131264,0.170877,0.768182
META,0.261924,0.351336,0.745509


# Optimization
$$
\begin{aligned}
\text{Minimize: } &\quad f(w) = w^T \Sigma w \\
\text{subject to: } &\quad \sum_i w_i = 1 \\
&\quad w^T \mu = \mu^*_{\text{target}} \\
&\quad -0.20 \le w_i \le 0.35
\end{aligned}
$$

In [22]:
# ========== Optimization ==========
x0 = np.ones(n) / n       # initial guess weight

# minimize portfolio variance
def obj_var_weekly(w, cov_mat):
    return float(w @ cov_mat @ w)

# The expected return of the portfolio must equal a given target rate of return
cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},
    {'type': 'eq', 'fun': lambda w, m=mean_weekly.values, t=target_mean_weekly: float(np.dot(w, m) - t)}
] 

bounds = tuple((LOWER, UPPER) for _ in range(n))

In [23]:
def port_stats(weights, mean_vec, cov_mat):
    w = np.array(weights)
    port_mean = w.dot(mean_vec)
    port_var = obj_var_weekly(w, cov_mat)
    port_std = np.sqrt(port_var)
    port_sharpe = (port_mean - RISK_FREE) / port_std if port_std > 0 else np.nan
    return port_mean, port_std, port_sharpe

In [24]:
# ========== Bounded 0ptimization ==========
res_bounded = minimize(
    fun=lambda w: obj_var_weekly(w, cov_weekly.values),
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=cons,
    options={'ftol':1e-12, 'maxiter':1000}
)

if not res_bounded.success:
    print('Bounded optimization warning:', res_bounded.message)

w_bounded = res_bounded.x
mean_bounded_weekly, std_bounded_weekly, sharpe_bounded = port_stats(w_bounded, mean_weekly.values, cov_weekly.values)
mean_bounded_annual = mean_bounded_weekly * FREQ
std_bounded_annual = std_bounded_weekly * np.sqrt(FREQ)

In [25]:
# ========== Unbounded 0ptimization ==========
res_unbounded = minimize(
    fun=lambda w: obj_var_weekly(w, cov_weekly.values),
    x0=x0,
    method='SLSQP',
    bounds=None,
    constraints=cons,
    options={'ftol':1e-12, 'maxiter':1000}
)

if not res_unbounded.success:
    print('Unbounded optimization warning:', res_unbounded.message)

w_unbounded = res_unbounded.x
mean_unbounded_weekly, std_unbounded_weekly, sharpe_unbounded = port_stats(w_unbounded, mean_weekly.values, cov_weekly.values)
mean_unbounded_annual = mean_unbounded_weekly * FREQ
std_unbounded_annual = std_unbounded_weekly * np.sqrt(FREQ)

### 1.1

- Report the weights of the constrained portfolio.
- Report the mean, volatility, and Sharpe ratio of the resulting portfolio.

In [26]:
# ========== 1.1 ==========
bounded_df = pd.DataFrame({
    'ticker': tickers,
    'weight_bounded': w_bounded,
    'asset_mean_annual': mean_annual.values,
    'asset_std_annual': asset_annual_std,
    'asset_sharpe': asset_sharpes.values
}).set_index('ticker')

print('Bounded weights:')
print(bounded_df['weight_bounded'].round(6))
print(f"Bounded portfolio annual mean: {mean_bounded_annual:.6f}")
print(f"Bounded portfolio annual vol: {std_bounded_annual:.6f}")
print(f"Bounded portfolio annual Sharpe (rf=0): {(mean_bounded_annual-RISK_FREE)/std_bounded_annual:.6f}")

Bounded weights:
ticker
AAPL     0.029590
AMZN     0.093384
AVGO     0.036158
BRK/B    0.350000
GOOGL    0.008812
LLY      0.213302
META     0.002377
MSFT     0.145289
NVDA    -0.013601
TSLA    -0.016153
SPY      0.150842
Name: weight_bounded, dtype: float64
Bounded portfolio annual mean: 0.200000
Bounded portfolio annual vol: 0.164888
Bounded portfolio annual Sharpe (rf=0): 1.212945


### 1.2

- Compare these weights to the assets’ Sharpe ratios and means.
- Do the most extreme positions also have the most extreme Sharpe ratios and means?
- Why?

In [27]:
# ========== 1.2 ==========
extreme_long = bounded_df['weight_bounded'].idxmax()
extreme_short = bounded_df['weight_bounded'].idxmin()

print(f"Most extreme long position: {extreme_long}, weight = {bounded_df.loc[extreme_long,'weight_bounded']:.4f}")
print(f"Most extreme short position: {extreme_short}, weight = {bounded_df.loc[extreme_short,'weight_bounded']:.4f}")

bounded_df

Most extreme long position: BRK/B, weight = 0.3500
Most extreme short position: TSLA, weight = -0.0162


,weight_bounded,asset_mean_annual,asset_std_annual,asset_sharpe
ticker,,,,
AAPL,0.029590,0.238714,0.276629,0.862938
AMZN,0.093384,0.293447,0.306000,0.958976
AVGO,0.036158,0.394854,0.375132,1.052574
BRK/B,0.350000,0.135025,0.190654,0.708223
GOOGL,0.008812,0.216800,0.279851,0.774698
LLY,0.213302,0.281542,0.282980,0.994919
META,0.002377,0.261924,0.351336,0.745509
MSFT,0.145289,0.261402,0.239980,1.089266
NVDA,-0.013601,0.645580,0.463283,1.393490


#### Why
- Extreme weights are influenced both by individual asset expected returns (means) and by the covariance structure with other assets.
- An asset with a high Sharpe may receive a large long position, but if it is highly correlated with others or cannot help reduce portfolio variance while meeting the mean constraint, it might not become extreme.
- Conversely, an asset with a modest Sharpe but low correlation to others can be useful for variance reduction and therefore receive larger weight.
- Bounds (-20% to 35%) also truncate what would be extreme weights in unconstrained optimization, so the observed extremes may be at the bounds rather than 'natural' unconstrained values.

### 1.3

- Compare the bounded portfolio weights to the unbounded portfolio weights (obtained from optimizing without the inequality constraints, keeping the equality constraints.)
- Report the mean, volatility, and Sharpe ratio of both.

In [28]:
# ========== 1.3 ==========
unbounded_df = pd.DataFrame({
    'ticker': tickers,
    'weight_unbounded': w_unbounded,
    'asset_mean_annual': mean_annual.values,
    'asset_std_annual': asset_annual_std,
    'asset_sharpe': asset_sharpes.values
}).set_index('ticker')

comparison = bounded_df[['weight_bounded']].join(unbounded_df[['weight_unbounded']])

print('Compare the bounded portfolio weights to the unbounded portfolio weights')
print(comparison)


print('Bounded portfolio stats:')
print(f"Annual mean = {mean_bounded_annual:.6f}, annual vol = {std_bounded_annual:.6f}, Sharpe = {(mean_bounded_annual-RISK_FREE)/std_bounded_annual:.6f}")
print('Unbounded portfolio stats:')
print(f"Annual mean = {mean_unbounded_annual:.6f}, annual vol = {std_unbounded_annual:.6f}, Sharpe = {(mean_unbounded_annual-RISK_FREE)/std_unbounded_annual:.6f}")

Compare the bounded portfolio weights to the unbounded portfolio weights
        weight_bounded  weight_unbounded
ticker                                  
AAPL          0.029590          0.029726
AMZN          0.093384          0.093729
AVGO          0.036158          0.036343
BRK/B         0.350000          0.373030
GOOGL         0.008812          0.009809
LLY           0.213302          0.211063
META          0.002377          0.002969
MSFT          0.145289          0.146163
NVDA         -0.013601         -0.014415
TSLA         -0.016153         -0.015346
SPY           0.150842          0.126930
Bounded portfolio stats:
Annual mean = 0.200000, annual vol = 0.164888, Sharpe = 1.212945
Unbounded portfolio stats:
Annual mean = 0.200000, annual vol = 0.164861, Sharpe = 1.213141
